In [ ]:
import numpy
import matplotlib.pyplot as plt

from pathlib import Path
from scipy.ndimage import gaussian_filter
from scipy.stats import pearsonr
from tifffile import imread
from time import perf_counter

In [ ]:
def plot_path(name):
    with Path(name).open() as f:
        data = f.read()
    
    data = [list(map(float, line.split())) for line in data.strip().split("\n")]
    assert [len(d) == len(data[0]) for d in data]
    
    data = numpy.asarray(data, dtype=numpy.float)
    
    print("Pcoeff, p", pearsonr(data[:, 1], data[:, 2]))    
    plt.plot(data[:, 0], data[:, 1], label="target")
    plt.plot(data[:, 0], data[:, 2], label="pred")
    plt.legend()


In [ ]:
plot_path("C:/repos/lnet/paths/path_test.txt")

In [ ]:
plot_path( "C:/repos/lnet/paths/path_test2.txt")

In [ ]:
plot_path("/g/kreshuk/beuttenm/repos/lensletnet/logs/eval/19-10-10_12-39_2eeff61_on_platy.p_c/result/test/paths/epoch0/path0.txt")
# plot_path("K:/beuttenm/repos/lensletnet/logs/eval/19-10-10_12-39_2eeff61_on_platy.p_c/result/test/paths/epoch0/path0.txt")

In [ ]:
plot_path("K:/beuttenm/repos/lensletnet/logs/eval/19-10-10_12-39_2eeff61_on_platy.p_c/result/test/paths/epoch0/path1.txt")

In [ ]:
def get_data(name):
    with Path(name).open() as f:
        data = f.read()
    
    data = [list(map(float, line.split(","))) for line in data.strip().split("\n")[1:]]
    assert [len(d) == len(data[0]) for d in data]
    
    data =  numpy.asarray(data, dtype=numpy.float32)
    data[:, 1] /= numpy.iinfo(numpy.uint16).max
    return data

def plot_custom_path(name, add_label=""):
    pred = get_data(name)
    assert "_pred.csv" in name
    tgt = get_data(name.replace("_pred.csv", "_tgt.csv"))
    pcoeff, p = pearsonr(tgt[:, 1], pred[:, 1])
    print("Pcoeff, p for", add_label, pcoeff, p)    

    plt.plot(tgt[:, 0], tgt[:, 1], label="target" + add_label)
    plt.plot(pred[:, 0], pred[:, 1], label="pred" + add_label + f" {pcoeff:.2f}")
    plt.legend()

In [ ]:
# plot_custom_path("C:/Users/fbeut/owncloud/lensletnet/platy/custom_paths/customx131y125z16_pred.csv", " z16")
# plot_custom_path("C:/Users/fbeut/owncloud/lensletnet/platy/custom_paths/customx131y125z17_pred.csv", " z17")

In [ ]:
# plot_custom_path("C:/Users/fbeut/owncloud/lensletnet/platy/custom_paths/customx175y109z15_pred.csv", " z15")
# plot_custom_path("C:/Users/fbeut/owncloud/lensletnet/platy/custom_paths/customx175y109z16_pred.csv", " z16")
# plot_custom_path("C:/Users/fbeut/owncloud/lensletnet/platy/custom_paths/customx175y109z17_pred.csv", " z17")
plot_custom_path("C:/Users/fbeut/owncloud/lensletnet/platy/custom_paths/customx175y109z18_pred.csv", " z18")

In [ ]:
def get_raw_data(name):
    paths = sorted(list(Path(name).glob("*.tif")))
    data = []
    for path in paths:
        data.append((imread(path.as_posix()) / numpy.iinfo(numpy.uint16).max).astype(numpy.float32))
    
    return numpy.asarray(data)

In [ ]:
pred = get_raw_data("/g/kreshuk/beuttenm/repos/lensletnet/logs/eval/19-10-10_12-39_2eeff61_on_platy.p_c/result/test/prediction")

In [ ]:
print(pred.shape)

In [ ]:
tgt = get_raw_data("/g/kreshuk/beuttenm/repos/lensletnet/logs/eval/19-10-10_12-39_2eeff61_on_platy.p_c/result/test/target")

In [ ]:
print(tgt.shape)

In [ ]:
plt.imshow(pred[0 ,0].max(0))

In [ ]:
plt.imshow(tgt[0 ,0].max(0))

In [ ]:
pred_smooth = gaussian_filter(pred, sigma=2)

In [ ]:
plt.imshow(pred_smooth[0 ,0].max(0))



In [ ]:
pred_sample = numpy.array(pred[:10])



In [ ]:
plt.imshow(gaussian_filter(pred_sample, sigma=3)[5, 0, 14])




In [ ]:
def bin(data):
    shape = data.shape
    assert len(shape) == 5
    assert shape[1] == 1
    assert all([os % 10 == 0 for os in shape[3:]])
    data = data[:, 0]
    
    out_shape = numpy.asarray(data.shape)
    out_shape[0:2] -= 2
    out_shape[2:] //= 10
    assert all([os > 0 for os in out_shape])
    
    step = 10
    xy = numpy.mean(numpy.stack([data[:, :, x0::step, y0::step] for x0 in range(step) for y0 in range(step)]), axis=0)
    z = numpy.sum(numpy.stack([0.25* xy[:, :-2], 0.5 * xy[:, 1:-1], 0.25 * xy[:, 2:]]), axis=0)
    t = numpy.sum(numpy.stack([0.25* z[:-2], 0.5 * z[1:-1], 0.25 * z[2:]]), axis=0)
    
    return t

            

In [ ]:
pred_binned = bin(pred[:, :, :, 4:-4, 4:-4])
pred_binned.shape

In [ ]:
tgt_binned = bin(tgt[:, :, :, 4:-4, 4:-4])
tgt_binned.shape

In [ ]:
roi = (slice(None), 13, 10, 10)
print(pearsonr(pred_binned[roi], tgt_binned[roi]))

In [ ]:
start = perf_counter()
prs = numpy.empty(tgt_binned.shape[1:], dtype=numpy.float32)
for z in range(tgt_binned.shape[1]):
    for y in range(tgt_binned.shape[2]):
        for x in range(tgt_binned.shape[3]):
            prs[z, y, x] = pearsonr(pred_binned[:, z, y, x], tgt_binned[:, z, y, x])[0]

print(perf_counter() - start)

In [ ]:
plt.imshow(prs[0])
plt.colorbar()

In [ ]:
plt.imshow(tgt_binned[0].max(0))

In [ ]:
prs.max()

In [ ]:
stds = tgt_binned.std(axis=0)
stds.shape

In [ ]:
stds.mean()

In [ ]:
mask = stds > .03
masked_prs = numpy.array(prs)
masked_prs[~mask] = numpy.nan
print('mean masked pr', numpy.nanmean(masked_prs))
plt.imshow(mask.max(axis=0))

In [ ]:
mask = stds > .05
masked_prs = numpy.array(prs)
masked_prs[~mask] = numpy.nan
print('mean masked pr', numpy.nanmean(masked_prs))
plt.imshow(mask.max(axis=0))

In [ ]:
msum = mask.sum()
print(msum, msum / numpy.prod(mask.shape))

In [ ]:
show = numpy.array(tgt_binned[0])
show[~mask] = 0
plt.imshow(show.max(axis=0))

In [ ]:
def plot_masked(data, mask):
    data = numpy.array(data)
    data[~mask] = numpy.nan
    plt.imshow(numpy.nanmax(data, axis=0))
    plt.colorbar()

In [ ]:
plot_masked(prs, mask)
plt.show()

In [ ]:
def plot_ts(tgt, pred):
    fig, ax_tgt = plt.subplots()
    ax_pred = ax_tgt.twinx()
    ax_tgt.set_xlabel("t")
    ax_tgt.set_ylabel("target")
    ax_pred.set_ylabel("prediction")
    
    tgt = numpy.array(tgt)
    pred = numpy.array(pred)
    add_label = ""
    lns = ax_tgt.plot(tgt, color="r", label="target" + add_label)
    lns += ax_pred.plot(pred, color="b", label="prediction" + add_label) # + f" {pcoeff:.2f}

    labs = [l.get_label() for l in lns]
    ax_tgt.legend(lns, labs)
    plt.show()

In [ ]:
for idx in range(0, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
for idx in range(1, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
for idx in range(2, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
for idx in range(50, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
def plot_hist(prs):
    fig, ax = plt.subplots()
    ax.yaxis.tick_right()
    plt.hist(prs, bins=20)

In [ ]:
plot_hist(prs.flatten())

In [ ]:
plot_hist(prs[mask])